# Running Semantic Segmentation inference on custom data

In this tutorial, we will cover how to run an inference on a pointcloud data in Open3D-ML. To accomplish that, we will take these steps:

1. Download the data *weights* file;
2. Set up `torch` and `numpy` libraries;
3. Create a `dataset` object and extract a sample from its `'test'` split;
4. Create and initialize `model` and `pipeline` objects;
5. Restore the `model` with data from the *weights* file;
6. Convert the custom pointcloud data into the specified format;
7. Run an inference on the sample data.


> **Note:** We will be using a sample `RandLANet` `SemanticKITTI` weight file which we need to:
>
> 1. Download for either *PyTorch* or *TensorFlow* from links below:
> > a. For *PyTorch*: https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth
> >
> > b. For *TensorFlow*: https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.zip
>
> 2. Place the downloaded `randlanet_semantickitti_202201071330utc.pth` file into `'Open3D-ML/docs/tutorial/notebook/'` subdirectory, or any other place and change the `ckpt_path` accordingly.
>
> For other model/dataset weight files, please check out https://github.com/isl-org/Open3D-ML#semantic-segmentation-1


An inference predicts the results based on the trained model.

> **Please see the [Training a semantic segmentation model using PyTorch](train_ss_model_using_pytorch.ipynb) and [Training a semantic segmentation model using TensorFlow](train_ss_model_using_tensorflow.ipynb) for training tutorials.**

While training, the model saves the checkpoint files every few epochs, in the *logs* directory. We use these trained weights to restore the model for inference.

Our first step in inference on a custom data implementation is to import `open3d.ml` and `numpy` libraries:


In [ ]:
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import numpy as np

We then create a checkpoint path pointing to the weights file we downloaded (generated at the end of  the Training stage):

(You can download any other weights using a link from model zoo (collection of weights for all combinations of model and dataset): https://github.com/isl-org/Open3D-ML#semantic-segmentation-1 )

In [ ]:
weights_url = 'https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.zip'
ckpt_path = './randlanet_semantickitti_202201071330utc.pth'
# from urllib.request import urlretrieve
# urlretrieve(weights_url, filename=ckpt_path)

Now, we define a `dataset`, `model`, and `pipeline` objects identical to how it was done in our previous *Training a semantic segmentation model* tutorials:

In [ ]:
# We define dataset (similar to train_ss_using_pytorch tutorial)
dataset = ml3d.datasets.SemanticKITTI(dataset_path='SemanticKITTI/',
                                      cache_dir='./logs/cache',
                                      training_split=['00'],
                                      validation_split=['01'],
                                      test_split=['01'])

# Initializing the model and pipeline
model = ml3d.models.RandLANet(in_channels=3)
pipeline = ml3d.pipelines.SemanticSegmentation(model)

Next, we restore the model with our weights file with `pipeline.load_ckpt()` method:

In [ ]:
# Load checkpoint using `load_ckpt` method (restoring weights for inference)
pipeline.load_ckpt(ckpt_path=ckpt_path)

Now, lets query the first pointcloud from the `test` split.

In [ ]:
test_data = dataset.get_split('test')
data = test_data.get_data(0)

Let's display what `data` contains:

In [ ]:
print(data)

For inference on custom data, you can convert your pointcloud into this format:

**Dictionary with keys {'point', 'feat', 'label'}**

If you already have the *ground truth labels*, you can add them to data to get accuracy and IoU (Intersection over Union). Otherwise, pass labels as `None`.

And now - the main topic of our tutorial - running an inference on the test data. You can call the `run_inference()` method with your data, - it will print *accuracy per class* and *Intersection over Union (IoU)* metrics. The last entry in the list is *mean accuracy* and *mean IoU*:

In [ ]:
# Running inference on test data
results = pipeline.run_inference(data)
# prints per class accuracy and IoU (Intersection over Union). Last entry is mean accuracy and mean IoU.
# We get several `nan` outputs for missing classes in the input data.

The `results` object will return dictionary of predicted labels and predicted probabilities per point:

In [ ]:
# Dictionary of predicted labels and predicted probabilities per class
results